In [0]:
%cd gpt-2
!pip3 install -r requirements.txt
!python3 download_model.py 345M
!pip install webvtt-py youtube_dl
!pip install -U youtube-dl

Cloning into 'gpt-2'...
remote: Enumerating objects: 310, done.
remote: Total 310 (delta 0), reused 0 (delta 0), pack-reused 310
Receiving objects: 100% (310/310), 4.40 MiB | 2.55 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/content/gpt-2
     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 604kB 24.7MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=e0cb8a9cd11c0b1380c9b9c7e82641f8544dbe0af74df6aedde9e47b8d5e0ef5
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533168 sha256=5e0b913b16ad0b89506b425fe787d590c528256507d141f6cf93b3aa03b88f2f
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation:

Fetching checkpoint: 1.00kit [00:00, 884kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 49.3Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 819kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:27, 51.2Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 7.77Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 62.3Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 51.3Mit/s]                                                       
     |████████████████████████████████| 1.8MB 13.6MB/s 
Requirement already up-to-date: youtube-dl in /usr/local/lib/python3.6/dist-packages (2019.11.5)


## Train GPT-2

### Training Data

In [0]:
download_training_data = True # required only if you're looking to retrain

if download_training_data:
  # YouTube URLs of full FIFa/PES matches with uninterrupted game commentary
  urls = [
      'https://www.youtube.com/watch?v=_QL2Vr-Rbhk',    'https://www.youtube.com/watch?v=vW2pn2LsrZU',    'https://www.youtube.com/watch?v=d18-39m9NoE',
      'https://www.youtube.com/watch?v=rzOddgKPPtI',    'https://www.youtube.com/watch?v=px4EAg0Vbg4',    'https://www.youtube.com/watch?v=-DQuJTTcbdw',
      'https://www.youtube.com/watch?v=3_B3smljvkQ',    'https://www.youtube.com/watch?v=3us3vUoLkac',    'https://www.youtube.com/watch?v=ol4WW_IVGOQ',
      'https://www.youtube.com/watch?v=2cLT4mTqWz4',    'https://www.youtube.com/watch?v=TPZPB2uCh8k',    'https://www.youtube.com/watch?v=JuDngE09WhE',
      'https://www.youtube.com/watch?v=9rRp2gDE7Yc',    'https://www.youtube.com/watch?v=hqrlgNP7E-0',    'https://www.youtube.com/watch?v=fXhf9Yh7tf8',
      'https://www.youtube.com/watch?v=jV3FUzCqQAQ',    'https://www.youtube.com/watch?v=uhBw9DO-z7A',    'https://www.youtube.com/watch?v=rczhz6xeIfQ'
  ]

  import webvtt
  import requests
  import os
  import youtube_dl
  import re

  if not os.path.exists('captions'):
      os.makedirs('captions')

  # final training data file for all videos combined    
  f_train = open("commentary_train.txt", "w")

  # download captions only for all urls
  for ix, url in enumerate(urls):
    ydl = youtube_dl.YoutubeDL({'writesubtitles': True, 'allsubtitles': True, 'writeautomaticsub': True})
    res = ydl.extract_info(url, download=False)
    if res['requested_subtitles'] and res['requested_subtitles']['en']:
      print('Grabbing vtt file from ' + res['requested_subtitles']['en']['url'])
      response = requests.get(res['requested_subtitles']['en']['url'], stream=True)

      f1 = open("captions/commentary{}.txt".format(ix), "w")
      f1.write(response.text)
      f1.close()
      if len(res['subtitles']) > 0:
        print('manual captions')
      else:
        print('automatic_captions')
    else:
      print('Youtube Video does not have any english captions')
      continue

    # convert downloaded webvtt file to plain text transcript
    vtt = webvtt.read("captions/commentary{}.txt".format(ix))
    transcript = ""

    lines = []
    for line in vtt:
        lines.extend(line.text.strip().splitlines())

    previous = None
    for line in lines:
        if line == previous:
           continue
        transcript += " " + line
        previous = line

    print(transcript)
    # replace [Music] and [Applause] keywords that appear in youtube captions before writing transcript
    f_train.write(re.sub('\[.*?\]','',transcript)+'\n')

  f_train.close()

[youtube] _QL2Vr-Rbhk: Downloading webpage


ERROR: Unable to download webpage: HTTP Error 429: Too Many Requests (caused by <HTTPError 429: 'Too Many Requests'>); please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.


DownloadError: ignored

## Model Training

In [0]:
!PYTHONPATH=src ./train.py --dataset /content/gpt-2/commentary_train.txt --model_name '345M'
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-11-14 10:48:45.313758: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-11-14 10:48:45.313990: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x17c9480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-11-14 10:48:45.314023: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-11-14 10:48:45.319549: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-14 10:48:45.475199: I tensorflow/stream_executor/cu

- Here, I'm just going to pull the model I've trained before from my Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!zip run1.zip ./checkpoint/run1/*
!cp run_2.zip /content/drive/My\ Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
	zip warning: name not matched: ./checkpoint/run1/*

zip error: Nothing to do! (run1.zip)
cp: cannot stat 'run_2.zip': No such file or directory


In [0]:
 !cp -r /content/drive/My\ Drive/checkpoint/run1/* /content/gpt-2/models/345M/

cp: cannot stat '/content/drive/My Drive/checkpoint/run1/*': No such file or directory


In [0]:
%cd models/345M/
!rm checkpoint
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1iTRezA2oe2B58wDx3FcZosqdcyD06Kwt' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1iTRezA2oe2B58wDx3FcZosqdcyD06Kwt" -O run1.zip && rm -rf /tmp/cookies.txt
!unzip run1.zip
%cd ../..

/content/gpt-2/models/345M
--2019-11-18 17:36:12--  https://docs.google.com/uc?export=download&confirm=xMYH&id=1iTRezA2oe2B58wDx3FcZosqdcyD06Kwt
Resolving docs.google.com (docs.google.com)... 173.194.79.138, 173.194.79.139, 173.194.79.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.79.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cqe647krvvtch33ohfc881t3m8b0m5dh/1574092800000/13944929934665395551/*/1iTRezA2oe2B58wDx3FcZosqdcyD06Kwt?e=download [following]
--2019-11-18 17:36:13--  https://doc-00-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cqe647krvvtch33ohfc881t3m8b0m5dh/1574092800000/13944929934665395551/*/1iTRezA2oe2B58wDx3FcZosqdcyD06Kwt?e=download
Resolving doc-00-5g-docs.googleusercontent.com (doc-00-5g-docs.googleusercontent.com)... 173.194.79.132, 2a00:1450:4013:c05::84
Connecting to doc-00-5g

In [0]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"



2019-11-18 17:36:50.268259: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-18 17:36:50.306197: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-18 17:36:50.306830: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2019-11-18 17:36:50.307113: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2019-11-18 17:36:50.308723: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2019-11-18 17:36:50.310424: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
201